<a href="https://colab.research.google.com/github/L-ansari/Natural-Language-Processing-tasks/blob/master/attentionLstmUpdated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:
print(torch.__version__)

1.7.1+cu101


In [3]:
import pandas as pd
from pathlib import Path

In [4]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [5]:
df_train = pd.read_csv(Path('/content/drive/My Drive/DSP/data/bachelor/CLPsych_train.csv'))


In [6]:
df_train.head()

,label,text
0,depression_ctrl,Game over RT : Life is so unpredictable. Be gr...
1,depression,Happy ! Guys: please check your balls monthly ...
2,ptsd_ctrl,You gotta enjoy the view RT : i want a boyfrie...
3,ptsd_ctrl,I love that I'm so chill about everything RT :...
4,depression_ctrl,Photo: lkfrannyfrano: hotshxxt: migzfromatx: s...


In [7]:
print(df_train.shape[0])

1146


In [8]:
print(df_train['label'][1])

depression


In [9]:
for i in range (0, (df_train.shape[0])):
    if (df_train['label'][i]=='depression' or  df_train['label'][i]=='depression_ctrl') :
        
        df_train['label'][i]= 'positive' 
    else:
        #False
        df_train['label'][i]= 'negative' 
    
df_train.head()


,label,text
0,positive,Game over RT : Life is so unpredictable. Be gr...
1,positive,Happy ! Guys: please check your balls monthly ...
2,negative,You gotta enjoy the view RT : i want a boyfrie...
3,negative,I love that I'm so chill about everything RT :...
4,positive,Photo: lkfrannyfrano: hotshxxt: migzfromatx: s...


In [10]:
df_test = pd.read_csv(Path('/content/drive/My Drive/DSP/data/bachelor/CLPsych_test.csv'))


In [11]:
for i in range (0, (df_test.shape[0])):
    if (df_test['label'][i]=='depression' or  df_test['label'][i]=='depression_ctrl') :
        
        df_test['label'][i]= 'positive' 
    else:
        #False
        df_test['label'][i]= 'negative' 
    
df_test.head()


,label,text
0,negative,: the is happening... the balance of life...te...
1,positive,: Going with . Look for my lipstick in early t...
2,negative,Fox News: Second wave of health plan cancellat...
3,negative,no. The needles are just different for shading...
4,negative,On holiday... : Where's your favorite place to...


In [12]:
#export the columns we need as a new file  "train" 
df = pd.DataFrame(df_train, columns = ['text', 'label']) 
df.to_csv('train', index=False)


In [13]:
#export the columns we need as a new file "test" 
dff = pd.DataFrame(df_test, columns = ['text', 'label']) 
dff.to_csv('test', index=False)


In [14]:
import torch
import torchtext
from torchtext import data
from torchtext.vocab import Vectors, GloVe
import torch.utils.data as data_utils
from torch.utils.data import TensorDataset, DataLoader

In [15]:
tokenize = lambda x: x.split()
TEXT = data.Field(sequential=True, tokenize=tokenize, lower=True, include_lengths=True, batch_first=True, fix_length=200)
LABEL = data.LabelField(sequential=False)


In [16]:
fields = [('text', TEXT), ('label', LABEL)]

In [17]:
import sys
import csv

csv.field_size_limit(sys.maxsize)

131072

In [18]:
train_data, test_data = data.TabularDataset.splits(
    path='/content/',
    train='train',
    test='test',
    format='csv',
    fields=fields,
    skip_header=True
)

In [19]:
print(train_data.shape)

<generator object Dataset.__getattr__ at 0x7fcbee241f50>


In [20]:
TEXT.build_vocab(train_data, vectors=GloVe(name='6B', dim=300))


.vector_cache/glove.6B.zip: 862MB [02:42, 5.30MB/s]                           
100%|█████████▉| 399980/400000 [01:00<00:00, 9392.35it/s]

In [21]:
LABEL.build_vocab(train_data)

In [22]:
word_embeddings = TEXT.vocab.vectors

In [23]:
word_embeddings = TEXT.vocab.vectors
print ("Length of Text Vocabulary: " + str(len(TEXT.vocab)))
print ("Vector size of Text Vocabulary: ", TEXT.vocab.vectors.size())
print ("Label Length: " + str(len(LABEL.vocab)))

Length of Text Vocabulary: 870001
Vector size of Text Vocabulary:  torch.Size([870001, 300])
Label Length: 2


In [24]:
print(type(train_data))
print(train_data.shape)

<class 'torchtext.data.dataset.TabularDataset'>
<generator object Dataset.__getattr__ at 0x7fcb918b3dd0>


In [25]:

train_data, valid_data = train_data.split(split_ratio=0.6) # Further splitting of training_data to create new training_data & validation_data
train_iter, valid_iter, test_iter = data.BucketIterator.splits((train_data, valid_data, test_data), batch_size=32, sort_key=lambda x: len(x.text), repeat=False, shuffle=True)

vocab_size = len(TEXT.vocab)

In [26]:
print(torchtext.__version__)

0.3.1


In [27]:
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
#from torch.autograd import Variable
import torch.optim as optim
import numpy as np

In [28]:
def clip_gradient(model, clip_value):
    params = list(filter(lambda p: p.grad is not None, model.parameters()))
    for p in params:
        p.grad.data.clamp_(-clip_value, clip_value)
    
def train_model(model, train_iter, epoch):
    #Remove any epoch loss or accuracy from the previous run
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.cuda()     
    optim = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), weight_decay=1e-5)
    steps = 0
    model.train()        #Set the model to training mode
    for idx, batch in enumerate(train_iter): #for each bacth in the iterator
        text = batch.text[0]
        target = batch.label.long()
        #target.requires_grad=True
        if torch.cuda.is_available():
            text = text.cuda()
            target = target.cuda()
        if (text.size()[0] is not 32):# One of the batch returned by BucketIterator has length different than 32.
            continue
        optim.zero_grad()         #1. Clean the old gradients
        prediction = model(text)  #2. Forward pass
        loss = loss_fn(prediction, target)    #3. Calculate the loss
        num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).float().sum()
        acc = 100.0 * num_corrects/len(batch)
        loss.backward()         #4. Perform gradient decent differentiation backwards 
        clip_gradient(model, 1e-1)
        optim.step()         # 5. Gradient step: updates the weights for the model
        steps += 1 
        
        if steps % 100 == 0:
            print (f'Epoch: {epoch+1}, Idx: {idx+1}, Training Loss: {loss.item():.4f}, Training Accuracy: {acc.item(): .2f}%')
        
        total_epoch_loss += loss.item()  # Aggregate the epoch loss and accuracy within a given bacth
        total_epoch_acc += acc.item()  #The item method returns a scaler
        
    return total_epoch_loss/len(train_iter), total_epoch_acc/len(train_iter) #return average epoch loss & accuracy


In [113]:
def eval_model(model, val_iter):
    #Remove any epoch loss or accuracy from the previous run
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.eval()               #Set the model to evaluation mode-->disables batch normalization & dropouts
    with torch.no_grad():       #   We don't update the weights & gradients are not reqiured-->prevent gradient calculations
        for idx, batch in enumerate(val_iter): #for each batch in the iterator
            text = batch.text[0]
            if (text.size()[0] is not 32):
                continue
            target = batch.label.long()
            #target = torch.autograd.target.long()
            #target.requires_grad = True
            if torch.cuda.is_available():
                text = text.cuda()
                target = target.cuda()
            prediction = model(text)
            loss = loss_fn(prediction, target)
            #From all the prediction how many were right
            num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).sum()
            acc = 100.0 * num_corrects/len(batch)
            total_epoch_loss += loss.item() # Aggregate the epoch loss and accuracy within a given bacth
            total_epoch_acc += acc.item()  #The item method returns a scaler

    return total_epoch_loss/len(val_iter), total_epoch_acc/len(val_iter) #return average epoch loss & accuracy

In [114]:
class LSTMClassifier(nn.Module):
	def __init__(self, batch_size, output_size, hidden_size, vocab_size, embedding_length, weights):
		super(LSTMClassifier, self).__init__()
		
		"""
		Arguments
		---------
		batch_size : Size of the batch which is same as the batch_size of the data returned by the TorchText BucketIterator
		output_size : 2 = (pos, neg)
		hidden_sie : Size of the hidden_state of the LSTM
		vocab_size : Size of the vocabulary containing unique words
		embedding_length : Embeddding dimension of GloVe word embeddings
		weights : Pre-trained GloVe word_embeddings which we will use to create our word_embedding look-up table 
		
		"""
		
		self.batch_size = batch_size
		self.output_size = output_size
		self.hidden_size = hidden_size
		self.vocab_size = vocab_size
		self.embedding_length = embedding_length
		
		self.word_embeddings = nn.Embedding(vocab_size, embedding_length)# Initializing the look-up table.
		self.word_embeddings.weight = nn.Parameter(weights, requires_grad=False) # Assigning the look-up table to the pre-trained GloVe word embedding.
		self.lstm = nn.LSTM(embedding_length, hidden_size)
		self.label = nn.Linear(hidden_size, output_size)
		
	def forward(self, input_sentence, batch_size=None):
	
		""" 
		Parameters
		----------
		input_sentence: input_sentence of shape = (batch_size, num_sequences)
		batch_size : default = None. Used only for prediction on a single sentence after training (batch_size = 1)
		
		Returns
		-------
		Output of the linear layer containing logits for positive & negative class which receives its input as the final_hidden_state of the LSTM
		final_output.shape = (batch_size, output_size)
		
		"""
		
		''' Here we will map all the indexes present in the input sequence to the corresponding word vector using our pre-trained word_embedddins.'''
		input = self.word_embeddings(input_sentence) # embedded input of shape = (batch_size, num_sequences,  embedding_length)
		input = input.permute(1, 0, 2) # input.size() = (num_sequences, batch_size, embedding_length)
		if batch_size is None:
			h_0 = torch.zeros(1, self.batch_size, self.hidden_size).cuda() # Initial hidden state of the LSTM
			c_0 = torch.zeros(1, self.batch_size, self.hidden_size).cuda() # Initial cell state of the LSTM
		else:
			h_0 = torch.zeros(1, batch_size, self.hidden_size).cuda()
			c_0 = torch.zeros(1, batch_size, self.hidden_size).cuda()
   
		output, (final_hidden_state, final_cell_state) = self.lstm(input, (h_0, c_0))
		final_output = self.label(final_hidden_state[-1]) # final_hidden_state.size() = (1, batch_size, hidden_size) & final_output.size() = (batch_size, output_size)
		
		return final_output


In [115]:
import torch
import torch.nn as nn
#from torch.autograd import Variable
from torch.nn import functional as F
import numpy as np

class AttentionModel(torch.nn.Module):
	def __init__(self, batch_size, output_size, hidden_size, vocab_size, embedding_length, weights):
		super(AttentionModel, self).__init__()
		
		"""
		Arguments
		---------
		batch_size : Size of the batch which is same as the batch_size of the data returned by the TorchText BucketIterator
		output_size : 2 = (pos, neg)
		hidden_sie : Size of the hidden_state of the LSTM
		vocab_size : Size of the vocabulary containing unique words
		embedding_length : Embeddding dimension of GloVe word embeddings
		weights : Pre-trained GloVe word_embeddings which we will use to create our word_embedding look-up table 
		
		--------
		
		"""
		
		self.batch_size = batch_size
		self.output_size = output_size
		self.hidden_size = hidden_size
		self.vocab_size = vocab_size
		self.embedding_length = embedding_length

		#Layers within model:  Embedding - LSTM  - Linear (Fully connected).
		self.word_embeddings = nn.Embedding(vocab_size, embedding_length)
		self.word_embeddings.weights = nn.Parameter(weights, requires_grad=False)
		self.lstm = nn.LSTM(embedding_length, hidden_size)
		self.label = nn.Linear(hidden_size, output_size)
		#self.attn_fc_layer = nn.Linear()
		
	def attention_net(self, lstm_output, final_state):

		""" 
		Attention is used to compute soft alignment score corresponding
		between each of the hidden_state and the last hidden_state of the LSTM. 
		
		Arguments
		---------
		
		lstm_output : Final output of the LSTM which contains hidden layer outputs for each sequence.
		final_state : Final time-step hidden state (h_n) of the LSTM
		
		---------
		
		Returns : It performs attention mechanism by first computing weights for each of the sequence present in lstm_output and and then finally computing the
				  new hidden state.
				  
		Tensor Size :
					hidden.size() = (batch_size, hidden_size)
					attn_weights.size() = (batch_size, num_seq)
					soft_attn_weights.size() = (batch_size, num_seq)
					new_hidden_state.size() = (batch_size, hidden_size)
					  
		"""
		
		hidden = final_state.squeeze(0)
		attn_weights = torch.bmm(lstm_output, hidden.unsqueeze(2)).squeeze(2)
		soft_attn_weights = F.softmax(attn_weights, 1)
		new_hidden_state = torch.bmm(lstm_output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2)
		
		return new_hidden_state
	
	def forward(self, input_sentences, batch_size=None):
	
		""" 
		Parameters
		----------
		input_sentence: input_sentence of shape = (batch_size, num_sequences)
		batch_size : default = None. Used only for prediction on a single sentence after training (batch_size = 1)
		
		Returns
		-------
		Output of the linear layer containing logits for pos & neg class which receives its input as the new_hidden_state which is basically the output of the Attention network.
		final_output.shape = (batch_size, output_size)
		
		"""
		#First: Embedding
		input = self.word_embeddings(input_sentences)
		input = input.permute(1, 0, 2)
		if batch_size is None:
			h_0 = torch.zeros(1, self.batch_size, self.hidden_size).cuda()
			c_0 = torch.zeros(1, self.batch_size, self.hidden_size).cuda()
		else:
			h_0 = torch.zeros(1, batch_size, self.hidden_size).cuda()
			c_0 = torch.zeros(1, batch_size, self.hidden_size).cuda()
		
    #Second: LSTM - response consists of output state and a tupple of (hidden state, cell state)
		output, (final_hidden_state, final_cell_state) = self.lstm(input, (h_0, c_0)) # final_hidden_state.size() = (1, batch_size, hidden_size) 
		output = output.permute(1, 0, 2) # output.size() = (batch_size, num_seq, hidden_size)
		#Third: Attention
		attn_output = self.attention_net(output, final_hidden_state)
    #Fourth linear fully connected
		logits = self.label(attn_output)
		
		return logits

In [116]:

class RNN(nn.Module):
	def __init__(self, batch_size, output_size, hidden_size, vocab_size, embedding_length, weights):
		super(RNN, self).__init__()

		"""
		Arguments
		---------
		batch_size : Size of the batch which is same as the batch_size of the data returned by the TorchText BucketIterator
		output_size : 2 = (pos, neg)
		hidden_sie : Size of the hidden_state of the LSTM
		vocab_size : Size of the vocabulary containing unique words
		embedding_length : Embeddding dimension of GloVe word embeddings
		weights : Pre-trained GloVe word_embeddings which we will use to create our word_embedding look-up table 
		
		"""

		self.batch_size = batch_size
		self.output_size = output_size
		self.hidden_size = hidden_size
		self.vocab_size = vocab_size
		self.embedding_length = embedding_length
		
		self.word_embeddings = nn.Embedding(vocab_size, embedding_length)
		self.word_embeddings.weight = nn.Parameter(weights, requires_grad=False)
		self.rnn = nn.RNN(embedding_length, hidden_size, num_layers=2, bidirectional=True)
		self.label = nn.Linear(4*hidden_size, output_size)
	
	def forward(self, input_sentences, batch_size=None):
		
		""" 
		Parameters
		----------
		input_sentence: input_sentence of shape = (batch_size, num_sequences)
		batch_size : default = None. Used only for prediction on a single sentence after training (batch_size = 1)
		
		Returns
		-------
		Output of the linear layer containing logits for pos & neg class which receives its input as the final_hidden_state of RNN.
		logits.size() = (batch_size, output_size)
		
		"""

		input = self.word_embeddings(input_sentences)
		input = input.permute(1, 0, 2)
		if batch_size is None:
			h_0 = torch.zeros(4, self.batch_size, self.hidden_size).cuda() # 4 = num_layers*num_directions
		else:
			h_0 = torch.zeros(4, batch_size, self.hidden_size).cuda()
		output, h_n = self.rnn(input, h_0)
		# h_n.size() = (4, batch_size, hidden_size)
		h_n = h_n.permute(1, 0, 2) # h_n.size() = (batch_size, 4, hidden_size)
		h_n = h_n.contiguous().view(h_n.size()[0], h_n.size()[1]*h_n.size()[2])
		# h_n.size() = (batch_size, 4*hidden_size)
		logits = self.label(h_n) # logits.size() = (batch_size, output_size)
		
		return logits


In [117]:
learning_rate = 2e-5
batch_size = 32
output_size = 2
hidden_size = 256
embedding_length = 300

#model = LSTMClassifier(batch_size, output_size, hidden_size, vocab_size, embedding_length, word_embeddings)
#model = AttentionModel(batch_size, output_size, hidden_size, vocab_size, embedding_length, word_embeddings)
model = RNN(batch_size, output_size, hidden_size, vocab_size, embedding_length, word_embeddings)


loss_fn = F.cross_entropy

In [118]:
CUDA_LAUNCH_BLOCKING=1

In [119]:
for epoch in range(20):

    #Forward pass- Calculate loss- Perform GD- Take gradient step- Zero gradients
    train_loss, train_acc = train_model(model, train_iter, epoch)
    val_loss, val_acc = eval_model(model, valid_iter)
    if epoch %5 == 4:  #print  only every 5 epochs
      print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}%, Val. Loss: {val_loss:3f}, Val. Acc: {val_acc:.2f}%')
    
test_loss, test_acc = eval_model(model, test_iter)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc:.2f}%')

''' Let us now predict the sentiment on a single sentence just for the testing purpose. '''
test_sen1 = "Life is good."
test_sen2 = "The end is close."

test_sen1 = TEXT.preprocess(test_sen1)
test_sen1 = [[TEXT.vocab.stoi[x] for x in test_sen1]]

test_sen2 = TEXT.preprocess(test_sen2)
test_sen2 = [[TEXT.vocab.stoi[x] for x in test_sen2]]

test_sen = np.asarray(test_sen1)
test_sen = torch.LongTensor(test_sen)
test_tensor = test_sen
test_tensor = test_tensor.cuda()
model.eval()


output = model(test_tensor, 1)
#Feed the output to softmax to get the prediction
out = F.softmax(output, 1)

if (torch.argmax(out[0]) == 1): 
    print ("Sentiment: Positive")
else:
    print ("Sentiment: Negative")


Epoch: 05, Train Loss: 0.419, Train Acc: 75.43%, Val. Loss: 1.025083, Val. Acc: 49.34%
Epoch: 10, Train Loss: 0.421, Train Acc: 78.23%, Val. Loss: 1.092520, Val. Acc: 47.37%
Epoch: 15, Train Loss: 0.271, Train Acc: 85.13%, Val. Loss: 1.321145, Val. Acc: 46.22%
Epoch: 20, Train Loss: 0.194, Train Acc: 88.69%, Val. Loss: 1.701713, Val. Acc: 46.05%
Test Loss: 1.495, Test Acc: 42.97%
Sentiment: Positive


In [68]:
from sklearn.model_selection import KFold
kfold =KFold(n_splits=10)

In [69]:
print(df.shape)

(1146, 2)


In [70]:
cv = KFold(n_splits=5, random_state=42, shuffle=False)
for train_index, test_index in cv.split(df):
    #print(df.iloc[train_index])
    #print(df.iloc[test_index])
    #print("Train Index: ", train_index, "\n")
    #print("Test Index: ", test_index)
    print("------")
    



------
------
------
------
------


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [103]:
train_loss_list = []
train_accuracy_list = []
validation_loss_list = []
validation_accuracy_list=[]

for fold, (train_index, test_index) in enumerate(cv.split(df)):
  #print(fold)
  train=df.iloc[train_index].to_csv(f'train{fold}', index=False)
  valid=df.iloc[test_index].to_csv(f'valid{fold}', index=False)

  train_data, test_data, valid_data = data.TabularDataset.splits(
  path='/content/',
  train=f'train{fold}',
  test='test',
  validation=f'valid{fold}',
  format='csv',
  fields=fields,
  skip_header=True
)
  
  #train_data, valid_data = train_data.split(split_ratio=0.6) # Further splitting of training_data to create new training_data & validation_data
  train_iter, valid_iter, test_iter = data.BucketIterator.splits((train_data, valid_data, test_data), batch_size=32, sort_key=lambda x: len(x.text), repeat=False, shuffle=True)

  print(f'Fold number: {fold}',"\n")

  
  for epoch in range(30):

    #Forward pass- Calculate loss- Perform GD- Take gradient step- Zero gradients
    train_loss, train_acc = train_model(model, train_iter, epoch)
    val_loss, val_acc = eval_model(model, valid_iter)

  
    if epoch %5 == 4: #print every 5 epochs

    
      print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}%, Val. Loss: {val_loss:3f}, Val. Acc: {val_acc:.2f}%')

  print("\n")
  train_loss_list.append(train_loss)
  train_accuracy_list.append(train_acc)
  validation_loss_list.append(val_loss)
  validation_accuracy_list.append(val_acc)

print("Average train loss:", np.average(train_loss_list))
print("Average train accuracy:", np.average(train_accuracy_list))
print("Average Validation loss:", np.average(validation_loss_list))
print("Average Validation error:", np.average(validation_accuracy_list))



print("\n")
test_loss, test_acc = eval_model(model, test_iter)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc:.2f}%')

Fold number: 0 

Epoch: 05, Train Loss: 0.049, Train Acc: 94.61%, Val. Loss: 4.434430, Val. Acc: 51.15%
Epoch: 10, Train Loss: 0.034, Train Acc: 96.01%, Val. Loss: 5.028644, Val. Acc: 50.66%
Epoch: 15, Train Loss: 0.037, Train Acc: 95.47%, Val. Loss: 4.484118, Val. Acc: 50.33%
Epoch: 20, Train Loss: 0.042, Train Acc: 94.83%, Val. Loss: 4.611390, Val. Acc: 52.30%
Epoch: 25, Train Loss: 0.041, Train Acc: 95.47%, Val. Loss: 4.395967, Val. Acc: 49.18%
Epoch: 30, Train Loss: 0.018, Train Acc: 96.01%, Val. Loss: 4.669544, Val. Acc: 49.67%


Fold number: 1 

Epoch: 05, Train Loss: 0.067, Train Acc: 94.61%, Val. Loss: 4.696566, Val. Acc: 49.51%
Epoch: 10, Train Loss: 0.026, Train Acc: 95.47%, Val. Loss: 4.126141, Val. Acc: 49.51%
Epoch: 15, Train Loss: 0.032, Train Acc: 95.26%, Val. Loss: 5.099572, Val. Acc: 48.52%
Epoch: 20, Train Loss: 0.053, Train Acc: 94.83%, Val. Loss: 5.080209, Val. Acc: 46.88%
Epoch: 25, Train Loss: 0.020, Train Acc: 96.01%, Val. Loss: 4.996573, Val. Acc: 51.81%
Epoch: 

In [104]:
len(validation_accuracy_list)

5

In [141]:
for idx, batch in enumerate(test_iter): #for each batch in the test iterator
  text = batch.text[0]
  if (text.size()[0] is not 32):
    continue

  target = batch.label.long()
            
  if torch.cuda.is_available():
    text = text.cuda()
    target = target.cuda()
  model.eval()
  prediction = model(text)
  loss = loss_fn(prediction, target)
  y_pred=target.data
  y_test=torch.max(prediction, 1)[1].view(target.size()).data
  num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).sum()
  

In [127]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics


In [145]:
print(metrics.classification_report(y_pred.cpu().detach().numpy(), y_test.cpu().detach().numpy()))

              precision    recall  f1-score   support

           0       0.43      0.75      0.55        12
           1       0.73      0.40      0.52        20

    accuracy                           0.53        32
   macro avg       0.58      0.57      0.53        32
weighted avg       0.62      0.53      0.53        32



##Testing with Reddit Dataset

In [152]:
reddit= pd.read_csv(Path('/content/drive/My Drive/DSP/data/bachelor/Reddit.csv'))
reddit.head(2)


,label,text
0,negative,My male cousin stopped talking to me all of su...
1,negative,Drifting away from my best friend\nMy best fri...


In [153]:
# Shuffle the reddit data set and split to train and test. Train will be later devided to train and validation. 
reddit= reddit.sample(frac=1).reset_index(drop=True)
reddit.head(10)


,label,text
0,positive,I’ve lost the last of my hope for “my turn”.\n...
1,positive,".\nI feel like I don't have a heart anymore, o..."
2,positive,A stranger hugged me yesterday\nI was crying y...
3,negative,Top 15 Inspiring Friendship Quotes\nA friend i...
4,positive,Kill myself because I don't have a girlfriend\...
5,positive,I've fallen back into a depression and missed ...
6,positive,Painfully lonely and spiraling into the worst ...
7,positive,When did you decide to get help? What was the ...
8,positive,3 in the night and I can't stop crying\nPeople...
9,negative,Is this my fault?\nI've been on SSI ever since...


In [154]:
reddit.shape[0]

1841

In [155]:
reddit_train=reddit.iloc[:1500] #the first 1500 rows are train
reddit_test=reddit.iloc[1500:] #the last 341 are test

In [156]:
reddit_train= pd.DataFrame(reddit_train, columns = ['text', 'label']) 
reddit_train.to_csv('reddit_train', index=False)

reddit_test= pd.DataFrame(reddit_test, columns = ['text', 'label']) 
reddit_test.to_csv('reddit_test', index=False)


In [157]:
train_data, test_data = data.TabularDataset.splits(
    path='/content/',
    train='reddit_train',
    test='reddit_test',
    format='csv',
    fields=fields,
    skip_header=True
)

In [158]:
TEXT.build_vocab(train_data, vectors=GloVe(name='6B', dim=300))


In [159]:
LABEL.build_vocab(train_data)

In [160]:
word_embeddings = TEXT.vocab.vectors

In [161]:
word_embeddings = TEXT.vocab.vectors
print ("Length of Text Vocabulary: " + str(len(TEXT.vocab)))
print ("Vector size of Text Vocabulary: ", TEXT.vocab.vectors.size())
print ("Label Length: " + str(len(LABEL.vocab)))

Length of Text Vocabulary: 21834
Vector size of Text Vocabulary:  torch.Size([21834, 300])
Label Length: 2


In [162]:

train_data, valid_data = train_data.split(split_ratio=0.6) # Further splitting of training_data to create new training_data & validation_data
train_iter, valid_iter, test_iter = data.BucketIterator.splits((train_data, valid_data, test_data), batch_size=32, sort_key=lambda x: len(x.text), repeat=False, shuffle=True)

vocab_size = len(TEXT.vocab)

In [163]:
learning_rate = 2e-5
batch_size = 32
output_size = 2
hidden_size = 256
embedding_length = 300

#model = LSTMClassifier(batch_size, output_size, hidden_size, vocab_size, embedding_length, word_embeddings)
#model = AttentionModel(batch_size, output_size, hidden_size, vocab_size, embedding_length, word_embeddings)
model = RNN(batch_size, output_size, hidden_size, vocab_size, embedding_length, word_embeddings)


loss_fn = F.cross_entropy

In [164]:
train_loss_list = []
train_accuracy_list = []
validation_loss_list = []
validation_accuracy_list=[]

for fold, (train_index, test_index) in enumerate(cv.split(df)):
  #print(fold)
  train=df.iloc[train_index].to_csv(f'train{fold}', index=False)
  valid=df.iloc[test_index].to_csv(f'valid{fold}', index=False)

  train_data, test_data, valid_data = data.TabularDataset.splits(
  path='/content/',
  train=f'train{fold}',
  test='test',
  validation=f'valid{fold}',
  format='csv',
  fields=fields,
  skip_header=True
)
  
  #train_data, valid_data = train_data.split(split_ratio=0.6) # Further splitting of training_data to create new training_data & validation_data
  train_iter, valid_iter, test_iter = data.BucketIterator.splits((train_data, valid_data, test_data), batch_size=32, sort_key=lambda x: len(x.text), repeat=False, shuffle=True)

  print(f'Fold number: {fold}',"\n")

  
  for epoch in range(30):

    #Forward pass- Calculate loss- Perform GD- Take gradient step- Zero gradients
    train_loss, train_acc = train_model(model, train_iter, epoch)
    val_loss, val_acc = eval_model(model, valid_iter)

  
    if epoch %5 == 4: #print every 5 epochs

    
      print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}%, Val. Loss: {val_loss:3f}, Val. Acc: {val_acc:.2f}%')

  print("\n")
  train_loss_list.append(train_loss)
  train_accuracy_list.append(train_acc)
  validation_loss_list.append(val_loss)
  validation_accuracy_list.append(val_acc)

print("Average train loss:", np.average(train_loss_list))
print("Average train accuracy:", np.average(train_accuracy_list))
print("Average Validation loss:", np.average(validation_loss_list))
print("Average Validation error:", np.average(validation_accuracy_list))



print("\n")
test_loss, test_acc = eval_model(model, test_iter)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc:.2f}%')

Fold number: 0 

Epoch: 05, Train Loss: 0.389, Train Acc: 79.09%, Val. Loss: 0.966876, Val. Acc: 53.12%
Epoch: 10, Train Loss: 0.223, Train Acc: 88.25%, Val. Loss: 1.611473, Val. Acc: 49.84%
Epoch: 15, Train Loss: 0.185, Train Acc: 89.76%, Val. Loss: 1.573199, Val. Acc: 50.99%
Epoch: 20, Train Loss: 0.309, Train Acc: 83.08%, Val. Loss: 1.338726, Val. Acc: 48.52%
Epoch: 25, Train Loss: 0.197, Train Acc: 89.33%, Val. Loss: 1.940569, Val. Acc: 47.20%
Epoch: 30, Train Loss: 0.102, Train Acc: 93.43%, Val. Loss: 2.454969, Val. Acc: 46.71%


Fold number: 1 

Epoch: 05, Train Loss: 0.166, Train Acc: 89.55%, Val. Loss: 1.694124, Val. Acc: 49.51%
Epoch: 10, Train Loss: 0.150, Train Acc: 90.62%, Val. Loss: 2.149475, Val. Acc: 49.18%
Epoch: 15, Train Loss: 0.130, Train Acc: 92.13%, Val. Loss: 2.352751, Val. Acc: 46.88%
Epoch: 20, Train Loss: 0.107, Train Acc: 92.89%, Val. Loss: 2.751747, Val. Acc: 48.03%
Epoch: 25, Train Loss: 0.068, Train Acc: 93.97%, Val. Loss: 2.913142, Val. Acc: 47.53%
Epoch: 

In [165]:
for idx, batch in enumerate(test_iter): #for each batch in the test iterator
  text = batch.text[0]
  if (text.size()[0] is not 32):
    continue

  target = batch.label.long()
            
  if torch.cuda.is_available():
    text = text.cuda()
    target = target.cuda()
  model.eval()
  prediction = model(text)
  loss = loss_fn(prediction, target)
  y_pred=target.data
  y_test=torch.max(prediction, 1)[1].view(target.size()).data
  num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).sum()
  

In [166]:
print(metrics.classification_report(y_pred.cpu().detach().numpy(), y_test.cpu().detach().numpy()))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92        12
           1       0.95      0.95      0.95        20

    accuracy                           0.94        32
   macro avg       0.93      0.93      0.93        32
weighted avg       0.94      0.94      0.94        32



##Testig with Mixed Dataset

In [167]:
Mixed= pd.read_csv(Path('/content/drive/My Drive/DSP/data/bachelor/Mixed.csv'))
Mixed.head(2)

,label,text
0,negative,My male cousin stopped talking to me all of su...
1,negative,Drifting away from my best friend\nMy best fri...


In [168]:
Mixed.shape[0]

2810

In [169]:
# Shuffle the reddit data set and split to train and test. Train will be later devided to train and validation. 
Mixed = Mixed.sample(frac=1).reset_index(drop=True)
Mixed.shape[0]


2810

In [170]:
Mixed_train=Mixed.iloc[:2500] #the first 2500 rows are train
Mixed_test=Mixed.iloc[2500:] #the last 310 are test

In [171]:
Mixed_test.shape[0]

310

In [172]:
Mixed_train= pd.DataFrame(Mixed_train, columns = ['text', 'label']) 
Mixed_train.to_csv('Mixed_train', index=False)

Mixed_test= pd.DataFrame(Mixed_test, columns = ['text', 'label']) 
Mixed_test.to_csv('Mixed_test', index=False)


In [173]:
train_data, test_data = data.TabularDataset.splits(
    path='/content/',
    train='Mixed_train',
    test='Mixed_test',
    format='csv',
    fields=fields,
    skip_header=True
)

In [174]:
TEXT.build_vocab(train_data, vectors=GloVe(name='6B', dim=300))


In [175]:
LABEL.build_vocab(train_data)

In [176]:
word_embeddings = TEXT.vocab.vectors

In [177]:
word_embeddings = TEXT.vocab.vectors
print ("Length of Text Vocabulary: " + str(len(TEXT.vocab)))
print ("Vector size of Text Vocabulary: ", TEXT.vocab.vectors.size())
print ("Label Length: " + str(len(LABEL.vocab)))

Length of Text Vocabulary: 35732
Vector size of Text Vocabulary:  torch.Size([35732, 300])
Label Length: 2


In [178]:

train_data, valid_data = train_data.split(split_ratio=0.6) # Further splitting of training_data to create new training_data & validation_data
train_iter, valid_iter, test_iter = data.BucketIterator.splits((train_data, valid_data, test_data), batch_size=32, sort_key=lambda x: len(x.text), repeat=False, shuffle=True)

vocab_size = len(TEXT.vocab)

In [179]:
learning_rate = 2e-5
batch_size = 32
output_size = 2
hidden_size = 256
embedding_length = 300

#model = LSTMClassifier(batch_size, output_size, hidden_size, vocab_size, embedding_length, word_embeddings)
#model = AttentionModel(batch_size, output_size, hidden_size, vocab_size, embedding_length, word_embeddings)
model = RNN(batch_size, output_size, hidden_size, vocab_size, embedding_length, word_embeddings)


loss_fn = F.cross_entropy

In [180]:
train_loss_list = []
train_accuracy_list = []
validation_loss_list = []
validation_accuracy_list=[]

for fold, (train_index, test_index) in enumerate(cv.split(df)):
  #print(fold)
  train=df.iloc[train_index].to_csv(f'train{fold}', index=False)
  valid=df.iloc[test_index].to_csv(f'valid{fold}', index=False)

  train_data, test_data, valid_data = data.TabularDataset.splits(
  path='/content/',
  train=f'train{fold}',
  test='test',
  validation=f'valid{fold}',
  format='csv',
  fields=fields,
  skip_header=True
)
  
  #train_data, valid_data = train_data.split(split_ratio=0.6) # Further splitting of training_data to create new training_data & validation_data
  train_iter, valid_iter, test_iter = data.BucketIterator.splits((train_data, valid_data, test_data), batch_size=32, sort_key=lambda x: len(x.text), repeat=False, shuffle=True)

  print(f'Fold number: {fold}',"\n")

  
  for epoch in range(30):

    #Forward pass- Calculate loss- Perform GD- Take gradient step- Zero gradients
    train_loss, train_acc = train_model(model, train_iter, epoch)
    val_loss, val_acc = eval_model(model, valid_iter)

  
    if epoch %5 == 4: #print every 5 epochs

    
      print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}%, Val. Loss: {val_loss:3f}, Val. Acc: {val_acc:.2f}%')

  print("\n")
  train_loss_list.append(train_loss)
  train_accuracy_list.append(train_acc)
  validation_loss_list.append(val_loss)
  validation_accuracy_list.append(val_acc)

print("Average train loss:", np.average(train_loss_list))
print("Average train accuracy:", np.average(train_accuracy_list))
print("Average Validation loss:", np.average(validation_loss_list))
print("Average Validation error:", np.average(validation_accuracy_list))



print("\n")
test_loss, test_acc = eval_model(model, test_iter)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc:.2f}%')

Fold number: 0 

Epoch: 05, Train Loss: 0.431, Train Acc: 75.97%, Val. Loss: 0.924748, Val. Acc: 48.03%
Epoch: 10, Train Loss: 0.309, Train Acc: 83.94%, Val. Loss: 1.206941, Val. Acc: 49.67%
Epoch: 15, Train Loss: 0.275, Train Acc: 84.81%, Val. Loss: 1.354936, Val. Acc: 48.03%
Epoch: 20, Train Loss: 0.156, Train Acc: 91.06%, Val. Loss: 1.830730, Val. Acc: 47.70%
Epoch: 25, Train Loss: 0.104, Train Acc: 92.03%, Val. Loss: 2.348845, Val. Acc: 49.34%
Epoch: 30, Train Loss: 0.096, Train Acc: 93.43%, Val. Loss: 2.099979, Val. Acc: 48.19%


Fold number: 1 

Epoch: 05, Train Loss: 0.196, Train Acc: 88.90%, Val. Loss: 1.835594, Val. Acc: 48.85%
Epoch: 10, Train Loss: 0.130, Train Acc: 91.27%, Val. Loss: 2.398595, Val. Acc: 48.52%
Epoch: 15, Train Loss: 0.087, Train Acc: 93.64%, Val. Loss: 2.531699, Val. Acc: 48.03%
Epoch: 20, Train Loss: 0.074, Train Acc: 93.64%, Val. Loss: 3.273916, Val. Acc: 48.19%
Epoch: 25, Train Loss: 0.057, Train Acc: 94.40%, Val. Loss: 3.421183, Val. Acc: 46.88%
Epoch: 

In [181]:
for idx, batch in enumerate(test_iter): #for each batch in the test iterator
  text = batch.text[0]
  if (text.size()[0] is not 32):
    continue

  target = batch.label.long()
            
  if torch.cuda.is_available():
    text = text.cuda()
    target = target.cuda()
  model.eval()
  prediction = model(text)
  loss = loss_fn(prediction, target)
  y_pred=target.data
  y_test=torch.max(prediction, 1)[1].view(target.size()).data
  num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).sum()
  

In [182]:
print(metrics.classification_report(y_pred.cpu().detach().numpy(), y_test.cpu().detach().numpy()))

              precision    recall  f1-score   support

           0       1.00      0.90      0.95        20
           1       0.86      1.00      0.92        12

    accuracy                           0.94        32
   macro avg       0.93      0.95      0.94        32
weighted avg       0.95      0.94      0.94        32

